### 中醫醫案症狀詞抽取

- model: CRF
- 資料集: 舌象
- 類別標註: OS
- 詞位標注: BIEO
- 拆句: 以句點拆分
- 斷詞: 沒有做
- 詞性標註: 使用中科院的
- 引用辭典: 沒有
- 標點: 沒去

In [1]:
import json
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [4]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    
    #原文
    case_content = cols[2]
    case_content = case_content.replace(" ","")
    case_content = case_content.replace('#',',')
    sentences = case_content.split("。")
    
    #摘要
    summary_str = cols[3].strip()
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "tongue" in keys:
        main = summary_json["tongue"]
        main_list = getTagListWithString(main)
        #print(main_list)
    
    t += len(main_list)

    for sent in sentences:
        ne_index_list = []
        for ne in main_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne)))

        texts = []
        for i,w in enumerate(sent):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))

        docs.append(texts)

print(len(docs))

48901


In [ ]:
for d in docs:
    if 

In [5]:
### 計算平均句長
total_wc = 0
for doc in docs:
    total_wc += len(doc)
    
ave = total_wc / 48901
print(ave)

32.537698615570235


In [6]:
import pynlpir  # 引入依赖包
pynlpir.open()  # 打开分词器

data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    
    tagged = []
    for tk in tokens:
        tg = pynlpir.segment(tk)
        tagged.append(tg[0])
    
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gm

In [7]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    
    if postag is None:
        postag = 'none'

    # Common features for all words
    features = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
        
    # Features for words that are not at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        
        if postag1 is None:
            postag1 = 'none'
        
        features.extend([
            '-1:word=' + word1,
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        
        if postag1 is None:
            postag1 = 'none'
        
        features.extend([
            '+1:word=' + word1,
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [8]:
# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
print(len(X_train))
print(len(X_test))

39120
9781


In [10]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 12291
Seconds required: 1.536

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 249701.324493
Feature norm: 1.000000
Error norm: 550735.312761
Active features: 12288
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 0.717

***** Iteration #2 *****
Loss: 137480.571107
Feature norm: 1.289275
Error norm: 231084.800282
Active features: 12204
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.366

***** Iteration #3 *****
Loss: 100702.086758
Feature norm: 1.499788
Error norm: 110425.132628
Active features: 12199
Line search trials: 1
Line search step: 1.000000
Seconds requir

***** Iteration #39 *****
Loss: 5959.750441
Feature norm: 24.721938
Error norm: 704.054291
Active features: 3663
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.366

***** Iteration #40 *****
Loss: 5869.442464
Feature norm: 25.970044
Error norm: 1428.931037
Active features: 3645
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.368

***** Iteration #41 *****
Loss: 5805.097574
Feature norm: 26.511305
Error norm: 1687.179054
Active features: 3618
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.375

***** Iteration #42 *****
Loss: 5742.822961
Feature norm: 27.631418
Error norm: 722.211035
Active features: 3576
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.367

***** Iteration #43 *****
Loss: 5687.668600
Feature norm: 28.678709
Error norm: 496.796847
Active features: 3590
Line search trials: 1
Line search step: 1.000000
Seconds requ

***** Iteration #82 *****
Loss: 4962.722901
Feature norm: 47.421257
Error norm: 704.561992
Active features: 2323
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.368

***** Iteration #83 *****
Loss: 4958.955330
Feature norm: 47.526876
Error norm: 256.360474
Active features: 2327
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.368

***** Iteration #84 *****
Loss: 4955.981750
Feature norm: 47.686062
Error norm: 235.049850
Active features: 2323
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.369

***** Iteration #85 *****
Loss: 4951.550014
Feature norm: 47.945245
Error norm: 341.215252
Active features: 2311
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.373

***** Iteration #86 *****
Loss: 4948.433526
Feature norm: 48.282802
Error norm: 708.925669
Active features: 2288
Line search trials: 1
Line search step: 1.000000
Seconds requir

***** Iteration #123 *****
Loss: 4894.700903
Feature norm: 52.747468
Error norm: 206.406198
Active features: 2092
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.369

***** Iteration #124 *****
Loss: 4894.591181
Feature norm: 52.838535
Error norm: 385.317219
Active features: 2090
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.367

***** Iteration #125 *****
Loss: 4893.700313
Feature norm: 52.868511
Error norm: 235.963629
Active features: 2085
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.366

***** Iteration #126 *****
Loss: 4893.418048
Feature norm: 52.962646
Error norm: 365.037757
Active features: 2079
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.366

***** Iteration #127 *****
Loss: 4892.606794
Feature norm: 52.999347
Error norm: 274.648203
Active features: 2079
Line search trials: 1
Line search step: 1.000000
Seconds r

***** Iteration #164 *****
Loss: 4870.420406
Feature norm: 55.048896
Error norm: 345.915567
Active features: 1988
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.369

***** Iteration #165 *****
Loss: 4869.770562
Feature norm: 55.076308
Error norm: 211.528599
Active features: 1990
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.367

***** Iteration #166 *****
Loss: 4869.558850
Feature norm: 55.135834
Error norm: 328.659417
Active features: 1985
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.369

***** Iteration #167 *****
Loss: 4868.886966
Feature norm: 55.160422
Error norm: 188.175881
Active features: 1982
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.368

***** Iteration #168 *****
Loss: 4868.650169
Feature norm: 55.205332
Error norm: 269.776668
Active features: 1982
Line search trials: 1
Line search step: 1.000000
Seconds r

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')

y_pred_train = [tagger.tag(xseq) for xseq in X_train]
y_pred_test = [tagger.tag(xseq) for xseq in X_test]

In [12]:
# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# training data F1-score
predictions_train = np.array([labels[tag] for row in y_pred_train for tag in row])
truths_train = np.array([labels[tag] for row in y_train for tag in row])
print("F1 score for training set:\n")
print(classification_report(
    truths_train, predictions_train,
    target_names=["O", "B-SYM","I-SYM","E-SYM"]))

print("\n")

# test data F1-score
predictions_test = np.array([labels[tag] for row in y_pred_test for tag in row])
truths_test = np.array([labels[tag] for row in y_test for tag in row])
print("F1 score for test set:\n")
print(classification_report(
    truths_test, predictions_test,
    target_names=["O", "B-SYM","I-SYM","E-SYM"]))

F1 score for training set:

             precision    recall  f1-score   support

          O       1.00      1.00      1.00   1262835
      B-SYM       0.68      0.85      0.75      3805
      I-SYM       0.67      0.89      0.77      5239
      E-SYM       0.67      0.85      0.75      3795

avg / total       1.00      0.99      0.99   1275674



F1 score for test set:

             precision    recall  f1-score   support

          O       1.00      1.00      1.00    312143
      B-SYM       0.65      0.80      0.72       977
      I-SYM       0.64      0.81      0.72      1357
      E-SYM       0.64      0.80      0.71       975

avg / total       0.99      0.99      0.99    315452

